In [1]:
import chaospy as cp
import numpy as np
import pandas as pd
import respy as rp
from os import getpid
from time import time
from joblib import Parallel, delayed

## Produce input parameters

In [4]:
# load model specifications
base_params, base_options = rp.get_example_model("kw_94_one", with_data=False)
# base_params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")
constraints = rp.get_parameter_constraints("kw_94_one")
# constraints.remove({"loc": "shocks_sdcorr", "type": "sdcorr"})
# mean and cov for sampling
mean = base_params["value"].to_numpy()[:27]
cov = pd.read_pickle("./input/respy-se-collection/covariance_kw_94_one.pkl").to_numpy()

# sample input parameters
np.random.seed(123)
distribution = cp.MvNormal(loc=mean, scale=cov)
input_params = distribution.sample(10).T # 10 draws for simplicit

input_params[0]

array([ 9.50316605e-01,  9.20652540e+00,  3.84247803e-02,  3.26048368e-02,
       -4.85946577e-04, -8.84486188e-04,  3.41632490e-05,  8.47219383e+00,
        7.02498326e-02,  6.75065851e-02, -1.01212486e-03,  2.18340379e-02,
       -4.65430956e-04, -1.22596487e+02, -1.54128078e+02, -4.22392916e+03,
        1.76453639e+04,  2.01213001e-01,  2.49539997e-01,  1.49999966e+03,
        1.43324422e+03, -1.20334279e+02,  7.64720641e+01, -1.04379407e+02,
       -2.68812097e+02, -1.08131134e+02, -7.89950094e+01])

## Transfer input parameters to respy fomat

In [6]:
def params_to_respy(input_params, *args):
    """transfer sampled paramters to respy format."""
    
    # baseline options and params for the indices.
    _, base_options = rp.get_example_model("kw_94_one", with_data=False)
    base_params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")

    params_idx = pd.Series(data=input_params, index=base_params.index[0:27])

    assert len(params_idx) == 27, "Length of KW94 vector must be 27."
    part_1 = params_idx

    rp_params, _ = rp.get_example_model("kw_94_one", with_data=False)
    part_2 = rp_params.iloc[27:31, 0]

    parts = [part_1, part_2]
    rp_params_series = pd.concat(parts)
    input_params_respy = pd.DataFrame(rp_params_series, columns=["value"])

    return input_params_respy

In [8]:
start = time()
input_params_respy = Parallel(n_jobs=-1)(delayed(params_to_respy)(i) for i in input_params)
end = time()
# print(params_idx_respy)
# print(f'\nTime to complete: {end - start:.2f}s\n')

input_params_respy[0]

value
category            name                                 
delta               delta                        0.950317
wage_a              constant                     9.206525
                    exp_edu                      0.038425
                    exp_a                        0.032605
                    exp_a_square                -0.000486
                    exp_b                       -0.000884
                    exp_b_square                 0.000034
wage_b              constant                     8.472194
                    exp_edu                      0.070250
                    exp_b                        0.067507
                    exp_b_square                -0.001012
                    exp_a                        0.021834
                    exp_a_square                -0.000465
nonpec_edu          constant                  -122.596487
                    at_least_twelve_exp_edu   -154.128078
                    not_edu_last_period      -4223.929161
nonpec_home         constant                 17645.363922
shocks_chol         chol_a                       0.201213
                    chol_b_a                     0.249540
                    chol_b                    1499.999661
                    chol_edu_a                1433.244217
                    chol_edu_b                -120.334279
                    chol_edu                    76.472064
                    chol_home_a               -104.379407
                    chol_home_b               -268.812097
                    chol_home_edu             -108.131134
                    chol_home                  -78.995009
lagged_choice_1_edu probability                  1.000000
initial_exp_edu_10  probability                  1.000000
maximum_exp         edu                         20.000000

## simulation

`contextmanager` assigning different path for different processor

In [9]:
import contextlib
import os
import shutil
from pathlib import Path
from time import time

@contextlib.contextmanager
def _temporary_working_directory():
    """Changes working directory and returns to previous on exit.
    The name of the temporary directory is 'temp_process-id_timestamp'
    The directory is deleted upon exit.


    """
    folder_name = f"temp_{os.getpid()}_{str(time()).replace('.', '')}"
    path = Path(".").resolve() / folder_name
    path.mkdir()
    prev_cwd = Path.cwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(prev_cwd)
        shutil.rmtree(path)

Define simulation function.


There seems no way to fix model parameters and reduce computational cost, because `get_simulate_func` and `simulate` take the whole parameter dataframe as input when perform the simulation

In [10]:
def simulation(input_params_respy):
    _, base_options = rp.get_example_model("kw_94_one", with_data=False)

    with _temporary_working_directory():
        simulate = rp.get_simulate_func(input_params_respy, base_options)
        df = simulate(input_params_respy)

    print("I'm process", os.getpid())

    return df

In [12]:
# with all processors
start = time()
df = Parallel(n_jobs=-1)(delayed(simulation)(params) for params in input_params_respy)
end = time()
print(f'\nTime to complete: {end - start:.2f}s\n')

/Users/yulei/opt/anaconda3/envs/kw94-replication/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



Time to complete: 29.95s



In [14]:
df[0]

Experience_A  Experience_B  Experience_Edu Lagged_Choice_1  \
Identifier Period                                                               
0          0                  0             0              10             edu   
           1                  0             0              11             edu   
           2                  0             1              11               b   
           3                  0             2              11               b   
           4                  0             2              12             edu   
...                         ...           ...             ...             ...   
999        35                 2            23              20               b   
           36                 3            23              20               a   
           37                 3            24              20               b   
           38                 4            24              20               a   
           39                 4            25              20               b   

                   Shock_Reward_A  Meas_Error_Wage_A  Shock_Reward_B  \
Identifier Period                                                      
0          0            -0.035035                  1        0.040965   
           1            -0.354560                  1        1.185316   
           2            -1.063705                  1        1.245234   
           3            -0.692603                  1       -1.238533   
           4            -0.943424                  1       -0.581919   
...                           ...                ...             ...   
999        35           -0.606851                  1       -0.653759   
           36           -0.902388                  1        0.079006   
           37           -0.127229                  1       -0.397838   
           38            1.281240                  1        0.746552   
           39            1.518037                  1        2.639648   

                   Meas_Error_Wage_B  Shock_Reward_Edu  Meas_Error_Wage_Edu  \
Identifier Period                                                             
0          0                       1         -0.713137                    1   
           1                       1          0.066755                    1   
           2                       1         -0.130574                    1   
           3                       1          0.768294                    1   
           4                       1          0.432645                    1   
...                              ...               ...                  ...   
999        35                      1         -0.640942                    1   
           36                      1          1.419669                    1   
           37                      1         -2.357019                    1   
           38                      1         -0.347854                    1   
           39                      1          1.101550                    1   

                   ...  Nonpecuniary_Reward_Edu  Wage_Edu  Flow_Utility_Edu  \
Identifier Period  ...                                                        
0          0       ...              -122.596487       NaN       -232.274977   
           1       ...              -122.596487       NaN       -768.296800   
           2       ...             -4346.525647       NaN      -6030.903652   
           3       ...             -4346.525647       NaN      -5131.404289   
           4       ...              -276.724565       NaN      -1525.770913   
...                ...                      ...       ...               ...   
999        35      ...                      NaN       NaN               NaN   
           36      ...                      NaN       NaN               NaN   
           37      ...                      NaN       NaN               NaN   
           38      ...                      NaN       NaN               NaN   
           39      ...                      NaN     

If `simulate` is outside the loop, `_temporary_working_directory` doesn't work because `simulate_func` can't find the path of state space.

In [17]:
simulate_func = rp.get_simulate_func(base_params, base_options)

In [36]:
from functools import partial

def simulation(input_params_respy, simulate_func):

    with _temporary_working_directory():
        df = simulate_func(input_params_respy)

    print("I'm process", os.getpid())

    return df

simulation = partial(simulation, simulate_func=simulate_func)

In [38]:
# with all processors
start = time()
df = Parallel(n_jobs=-1)(delayed(simulation)(params) for params in input_params_respy)
end = time()
print(f'\nTime to complete: {end - start:.2f}s\n')

PicklingError: Could not pickle the task to send it to the workers.